In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# 1. Đọc dữ liệu đã làm sạch
file_path = "./dataset/Amazon_Product_Review_cleaned.csv"
df = pd.read_csv(file_path)


# 3. Phân chia tập dữ liệu
X = df['review_body']
y = df['star_rating']

# 4. Xử lý giá trị NaN trong cột review_body
X = X.fillna('')

# Chuyển đổi văn bản thành đặc trưng số với TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)  # Giới hạn 5000 từ phổ biến nhất
X_tfidf = vectorizer.fit_transform(X)

# 5. Chia tập huấn luyện & kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# 6. Huấn luyện mô hình SVM với GridSearchCV
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm = GridSearchCV(SVC(), param_grid, cv=5)
svm.fit(X_train, y_train)

# 7. Dự đoán trên tập kiểm tra
y_pred = svm.best_estimator_.predict(X_test)

# 8. Đánh giá mô hình
print("Độ chính xác:", accuracy_score(y_test, y_pred))
print("\nMa trận nhầm lẫn:\n", confusion_matrix(y_test, y_pred))
print("\nBáo cáo phân loại:\n", classification_report(y_test, y_pred))

In [ ]:

# Danh sách 10 testcase demo:
test_sentences = [
    "I absolutely love this product, it's amazing!",  # 100% tiếng Anh, kỳ vọng: 5 sao
    "Tôi rất thích sản phẩm này",                      # 100% tiếng Việt, kỳ vọng: 5 sao
    "Not good at all, totally disappointing.",          # 100% tiếng Anh, kỳ vọng: 1 sao
    "Sản phẩm này không tốt chút nào",                  # 100% tiếng Việt, kỳ vọng: 1 sao
    "The product is okay, nothing special.",            # 100% tiếng Anh, kỳ vọng: 3 sao
    "Sản phẩm khá ổn, nhưng cần cải thiện.",            # 100% tiếng Việt, kỳ vọng: 3 sao
    "Excellent quality and outstanding performance.",   # 100% tiếng Anh, kỳ vọng: 5 sao
    "Chất lượng trung bình, không đáng giá.",          # 100% tiếng Việt, kỳ vọng: 2-3 sao
    "I love it, nhưng giá hơi cao.",                    # Mix tiếng Anh - tiếng Việt, kỳ vọng: 4 sao
    "Worst product ever, tệ quá, không mua lần nữa."     # Mix tiếng Anh - tiếng Việt, kỳ vọng: 1 sao
]

# Biểu diễn các câu test bằng TF-IDF
X_test_sentences = vectorizer.transform(test_sentences)
y_test_pred = svm.best_estimator_.predict(X_test_sentences)

# Hiển thị kết quả
for text, pred in zip(test_sentences, y_test_pred):
    print(f"Review: {text} -> Dự đoán: {pred} sao")